# Redes Origen-Destino

En este notebook exploraremos los viajes de la Encuesta Origen-Destino 2012 de Santiago utilizando visualizaciones de redes.


## Preámbulo y Carga de Datos

In [ ]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

load_dotenv()

AVES_ROOT = Path(os.environ['AVES_ROOT'])
EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"

In [ ]:
CENSUS_GEO_ROOT = Path(os.environ['CENSUS_GEO_ROOT'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd

from aves.config import setup_style

from aves.data import eod
from aves.data.census.loading import read_census_map
from aves.features.utils import normalize_rows

setup_style()

In [ ]:
comunas = read_census_map('comuna', path=CENSUS_GEO_ROOT / "R13")

In [ ]:
zones = gpd.read_file(AVES_ROOT / 'data' / 'processed' / 'scl_zonas_urbanas.json').set_index('ID')
zones.head()

In [ ]:
viajes = eod.read_trips(EOD_PATH)

# descartamos sectores que no sean relevantes en los orígenes y destinos de los viajes
viajes = viajes[
    (viajes["SectorOrigen"] != "Exterior a RM")
    & (viajes["SectorDestino"] != "Exterior a RM")
    & (viajes["SectorOrigen"] != "Extensión Sur-Poniente")
    & (viajes["SectorDestino"] != "Extensión Sur-Poniente")
    & pd.notnull(viajes["SectorOrigen"])
    & pd.notnull(viajes["SectorDestino"])
]

print(len(viajes))


In [ ]:
personas = eod.read_people(EOD_PATH)
viajes_persona = viajes.merge(personas)

In [ ]:
viajes_persona["Peso"] = (
    viajes_persona["FactorExpansion"] * viajes_persona["FactorPersona"]
)


## ¿Cómo se relacionan las comunas de acuerdo a los viajes entre ellas, por propósito?

Primero, preparemos el `GeoDataFrame` de comunas. Tenemos que quedarnos solo con las comunas que nos interesan, y tenemos que asegurarnos que tenga los mismos nombres que en el `DataFrame` de viajes.

In [ ]:
#zones.head()

Hacemos dos cosas:

- Como tenemos las zonas urbanas, filtramos el `GeoDataFrame` para quedarnos solamente con aquellas comunas que están en el `DataFrame` de zonas.
- Hacemos un diccionario de `código de comuna -> nombre de comuna` a partir de las zonas y lo aplicamos.

In [ ]:
comunas_urbanas = comunas[comunas['COMUNA'].isin(zones['Com'].unique())].drop('NOM_COMUNA', axis=1).copy()
comunas_urbanas['NombreComuna'] = comunas_urbanas['COMUNA'].map(dict(zip(zones['Com'], zones['Comuna'])))
comunas_urbanas.plot(facecolor="none", edgecolor="#abacab")

El mapa es demasiado grande, así que lo recortaremos utilizando las zonas que conocemos:

In [ ]:
from aves.features.geo import clip_area_geodataframe

In [ ]:
bounding_box = zones.total_bounds
bounding_box

In [ ]:
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=0.05)

Calculamos la lista de aristas de nuestra red, es decir, la cantidad de viajes de una comuna a otra. En este caso, lo haremos con los viajes al trabajo.

In [ ]:
matriz = (
    viajes_persona[
        (viajes_persona["Proposito"] == "Al trabajo")
        & (viajes_persona["ComunaOrigen"].isin(comunas_urbanas["NombreComuna"]))
        & (viajes_persona["ComunaDestino"].isin(comunas_urbanas["NombreComuna"]))
    ]
    .groupby(["ComunaOrigen", "ComunaDestino"])
    .agg(n_viajes=("Peso", "sum"))
    .reset_index()
)

matriz.head()


Podemos convertir esta lista en una matriz de adyacencia. Veamos como luce esta matriz con el esquema `adjacency_matrix`. Como vimos en clase, utiliza la misma codificación visual que el `heatmap` de tablas, por lo que podemos usar `seaborn.heatmap` para visualizarla:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(
    matriz.set_index(["ComunaOrigen", "ComunaDestino"])["n_viajes"]
    .unstack(fill_value=0)
    .pipe(normalize_rows),
    cmap="inferno_r",
    linewidth=1,
)


También podemos utilizar el dataframe `matriz` como una lista de aristas que podemos visualizar con un gráfico NodeLink. A diferencia de la clase pasada, donde debíamos calcular la posición de cada nodo, al utilizar información geográfica los nodos ya tienen una posición.

In [ ]:
from aves.models.network import Network

od_network = Network.from_edgelist(
    # graficamos los viajes más representativos
    matriz[(matriz["n_viajes"] > matriz["n_viajes"].quantile(0.75)) & (matriz['ComunaOrigen'] != matriz['ComunaDestino'])],
    source="ComunaOrigen",
    target="ComunaDestino",
    weight="n_viajes",
)


In [ ]:
matriz

In [ ]:
from aves.visualization.networks import NodeLink

nodelink = NodeLink(od_network)
nodelink.layout_nodes(method='geographical', geodataframe=comunas_urbanas, node_column='NombreComuna')

En esta ocasión tenemos una red dirigida. Sabemos que los viajes van desde una comuna de origen hasta una comuna de destino. Por tanto, necesitamos una manera de identificar la dirección de las aristas. Una manera de hacerlo es pintarlas con un gradiente de color donde el azul representa el origen y el rojo representa el destino:

In [ ]:
nodelink.set_edge_drawing('origin-destination')

In [ ]:
od_network.detect_communities(
    method="ranked", hierarchical_covariate_type="discrete-poisson"
)


In [ ]:
nodelink.set_node_drawing("plain", weights='in_degree', categories='community')


In [ ]:
nodelink.set_edge_drawing(method="weighted", curved=True, k=10)


In [ ]:
from aves.visualization.figures import figure_from_geodataframe

fig, ax = figure_from_geodataframe(zones, height=7)

# contexto
zones.plot(ax=ax, facecolor='#efefef', edgecolor='white', zorder=0)
comunas_urbanas.plot(ax=ax, facecolor='none', edgecolor='#abacab', zorder=1)

nodelink.plot(ax, nodes=dict(palette='PuRd', edgecolor='black', node_size=150, alpha=0.95), edges=dict(alpha=0.5), zorder=2)

ax.set_title('Viajes al trabajo en Santiago (en días laborales, EOD 2012)')

fig.tight_layout()

In [ ]:
[a[0] == a[1] for a in zip(range(10), range(10))]

En comparación con la matriz de adyacencia, en esta visualización además de identificar las relaciones entre comunas podemos apreciar el contexto geográfico. Antes de elegir una de estas dos visualizaciones, debemos considerar lo siguiente:

- ¿Nos interesa conocer la relación geográfica entre orígenes y destinos? Por ej., ¿queremos saber si comunas vecinas se comportan similar?¿Nos interesa la distancia de los viajes? En este caso, `node_link` es una buena solución.
- ¿Necesitamos ver _todas_ las aristas? Si es así, el gráfico de `node_link` podría ser inadecuado, ya que no podemos cambiar la posición de los nodos, ni podemos ver las aristas en las que el origen y destino son iguales. Debemos usar `adjacency_matrix`.

**Ejercicio Propuesto**: realizar el mismo análisis para distintos propósitos de viaje. Realizarlo a nivel de zonas (ver README de aves para un ejemplo con zonas).

In [ ]:
viajes_persona['ModoDifusion']

In [ ]:
matriz_zonas = (viajes_persona[(viajes_persona["Proposito"].isin(["Al trabajo", 'Al estudio'])) & (viajes_persona['ZonaOrigen'] != viajes_persona['ZonaDestino'])
                             & (viajes_persona['ZonaOrigen'].isin(zones.index))
                             & (viajes_persona['ZonaDestino'].isin(zones.index))
                             & (viajes_persona['ModoDifusion'] == 'Bip!')]
                    .groupby(['ComunaOrigen', 'ZonaOrigen', 'ZonaDestino'])
                    .agg(n_viajes=('Peso', 'sum'))
                    .sort_values('n_viajes', ascending=False)
                    .assign(cumsum_viajes=lambda x: x['n_viajes'].cumsum())
                    .assign(cumsum_viajes=lambda x: x['cumsum_viajes'] / x['cumsum_viajes'].max())
                    .reset_index()
)

matriz_zonas.head()

In [ ]:
zone_od_network = Network.from_edgelist(
    matriz_zonas[matriz_zonas['cumsum_viajes'] <= 0.5], source="ZonaOrigen", target="ZonaDestino", weight="n_viajes"
)#.largest_connected_component(directed=True)
zone_od_network.network, zone_od_network.num_vertices, zone_od_network.num_edges

In [ ]:
merged_zones = zones.reset_index().dissolve('ID')

In [ ]:
zone_nodelink = NodeLink(zone_od_network)
zone_nodelink.layout_nodes(method="geographical", geodataframe=merged_zones)
zone_nodelink.set_node_drawing("plain", weights='in_degree')
zone_nodelink.set_edge_drawing(method="origin-destination")

In [ ]:
merged_zones

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

# contexto
zones.plot(ax=ax, facecolor='#efefef', edgecolor='white', zorder=0)
comunas_urbanas.plot(ax=ax, facecolor='none', edgecolor='#abacab', zorder=1)

zone_nodelink.plot(ax, nodes=dict(color='white', edgecolor='black', node_size=150, alpha=0.95), edges=dict(alpha=0.5), zorder=2)

ax.set_title('Viajes al trabajo en Santiago (en días laborales, EOD 2012)')

fig.tight_layout()

In [ ]:
zone_nodelink.bundle_edges(
     method="force-directed", K=1, S=0.005, I=6, compatibility_threshold=0.65, C=6
)

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

# contexto
zones.plot(ax=ax, facecolor='#efefef', edgecolor='white', zorder=0)
comunas_urbanas.plot(ax=ax, facecolor='none', edgecolor='#abacab', zorder=1)

zone_nodelink.plot(ax, nodes=dict(color='white', edgecolor='black', node_size=50, alpha=0.95), edges=dict(alpha=0.5), zorder=2)

ax.set_title('Viajes al trabajo en Santiago')

fig.tight_layout()

In [ ]:
viajes_persona['DondeEstudia'].value_counts()

In [ ]:
viajes_persona.columns

In [ ]:
study_network = Network.from_edgelist(
    viajes_persona[
        (viajes_persona["Proposito"] == "Al estudio")
        & (viajes_persona["ZonaOrigen"] != viajes_persona["ZonaDestino"])
        #& (viajes_persona["ComunaOrigen"] == "La Pintana")
        & (viajes_persona["ZonaOrigen"].isin(zones.index))
        & (viajes_persona["ZonaDestino"].isin(zones.index))
        & (viajes_persona["DondeEstudia"].between(6, 8))
    ]
    .groupby(["ComunaOrigen", "ZonaOrigen", "ZonaDestino"])
    .agg(n_viajes=("Peso", "sum"))
    .sort_values("n_viajes", ascending=False)
    .assign(cumsum_viajes=lambda x: x["n_viajes"].cumsum())
    .assign(cumsum_viajes=lambda x: x["cumsum_viajes"] / x["cumsum_viajes"].max())
    .reset_index()
    .pipe(lambda x: x[x['cumsum_viajes'] <= 0.9]),
    source="ZonaOrigen",
    target="ZonaDestino",
    weight="n_viajes",
)
study_network

In [ ]:
la_pintana_zones = zones[zones['Comuna'] == 'La Pintana'].index
la_pintana_zones

In [ ]:
from graph_tool.search import BFSVisitor, bfs_search, StopSearch

class Visitor(BFSVisitor):
    def __init__(self, edge_filter):
        self.pred = study_network.graph.new_vertex_property("int64_t")
        self.dist = study_network.graph.new_vertex_property("int")
        self.edge_filter = edge_filter

    def discover_vertex(self, u):
        #print("-->", u, "has been discovered!")
        pass

    def examine_vertex(self, u):
        #print(u, "has been examined...")
        pass

    def tree_edge(self, e):
        self.pred[e.target()] = int(e.source())
        self.dist[e.target()] = self.dist[e.source()] + 1
        #print('dist', self.dist[e.target()])
        if self.dist[e.target()] == 1:
            self.edge_filter[e] = True
        else:
            raise StopSearch()

edge_filter = study_network.graph.new_edge_property('bool', False)
for node_id in la_pintana_zones:
    if node_id in study_network.node_map:
        bfs_search(study_network.graph, study_network.node_map[node_id], Visitor(edge_filter))
sum(edge_filter.a)

In [ ]:
la_pintana_study = study_network.subgraph(edge_filter=edge_filter)
la_pintana_study.graph

In [ ]:
other_study = study_network.subgraph(
    edge_filter=study_network.graph.new_edge_property(
        "bool", ~edge_filter.a.astype(bool)
    )
)
other_study.graph

In [ ]:
other_nodelink = NodeLink(other_study)
other_nodelink.layout_nodes(method="geographical", geodataframe=merged_zones)
other_nodelink.set_node_drawing("plain", weights='in_degree')
other_nodelink.set_edge_drawing(method="weighted", k=5)
other_nodelink.bundle_edges(
     method="force-directed", K=1, S=0.005, I=6, compatibility_threshold=0.65, C=6
)

In [ ]:
subgraph_nodelink = NodeLink(la_pintana_study)
subgraph_nodelink.layout_nodes(method="geographical", geodataframe=merged_zones)
subgraph_nodelink.set_node_drawing("plain", weights='in_degree')
subgraph_nodelink.set_edge_drawing(method="origin-destination")
subgraph_nodelink.bundle_edges(
     method="force-directed", K=1, S=0.005, I=6, compatibility_threshold=0.65, C=6
)

In [ ]:
comunas_urbanas[comunas_urbanas['NombreComuna'] == 'La Pintana']

In [ ]:
from aves.visualization.maps import add_basemap

geocontext = zones.loc[study_network.node_map.keys()]
fig, ax = figure_from_geodataframe(geocontext, height=7)

add_basemap(ax, AVES_ROOT / "data" / "processed" / "scl_positron_12_balanced.tif", zones.loc[study_network.node_map.keys()])

# contexto
#zones.plot(ax=ax, facecolor='#efefef', edgecolor='white', zorder=0)
comunas_urbanas[comunas_urbanas['NombreComuna'] == 'La Pintana'].plot(ax=ax, facecolor='none', edgecolor='#333333', linewidth=2, zorder=1)

other_nodelink.plot(ax, nodes=dict(color='white', edgecolor='black', node_size=50, alpha=0.95), edges=dict(alpha=0.5, linewidth=0.5), zorder=2)

subgraph_nodelink.plot(ax, nodes=dict(color='white', edgecolor='black', node_size=50, alpha=0.95), edges=dict(alpha=0.75, linewidth=1.5), zorder=2)

ax.set_title('Viajes a educación superior desde La Pintana')

fig.tight_layout()